In [1]:
import torch
from torch import cuda
from torch import device

In [2]:
'cuda:0' if cuda.is_available() else 'cpu'

'cuda:0'

In [3]:
device = device('cuda:0' if cuda.is_available() else 'cpu')


## VGG 16

In [4]:
from torchvision.models import vgg16

In [5]:
vgg = vgg16(pretrained=True)
print(vgg)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 118MB/s]  


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
!pip install torchsummary

In [7]:
from torch import nn
from torchsummary import summary

In [8]:
class FeatureExtractor(nn.Module):

  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
    # Extract Feature Layers
    self.features = model.features
    # Extract Average Pooling Layer
    self.pooling = model.avgpool
    # Convert the image into one-dimensional vector
    self.flatten = nn.Flatten()
  
  def forward(self, x):
    # Take image x and return a feature vector
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    return out 

feature_extractor = FeatureExtractor(vgg)
feature_extractor = feature_extractor.to(device)

In [9]:
print(summary(feature_extractor, input_size=(3, 224, 224)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [10]:
import cv2
from torchvision import transforms
from torch import no_grad

## Fruits and Vegetables

In [11]:
!pip install imutils

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25834 sha256=5b7537d305bc21fe917199eab746417ecb0eba74fa7f34ce7bb5b1b494d02d5c
  Stored in directory: /root/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils


In [12]:
import tarfile
from imutils import paths
import os
import cv2
import numpy as np
from torchvision import transforms
import pandas as pd

In [13]:
train_path = "/kaggle/input/fruit-and-vegetable-image-recognition"
train_path = "/kaggle/input/fruit-and-vegetable-image-recognition/train"
test_path = "/kaggle/input/fruit-and-vegetable-image-recognition/test"
val_path = "/kaggle/input/fruit-and-vegetable-image-recognition/validation"

In [14]:
datasetPath = '/kaggle/input/fruit-and-vegetable-image-recognition/train'

def loadDataset():
    imageList = []
    
    directories = [f for f in os.listdir(datasetPath) if os.path.isdir(os.path.join(datasetPath, f))]
    
    labelList = []
    label = -1

    for directory in directories:
        label = str(directory)
        # path = resized+'/'+directory
        path = os.path.join(datasetPath, directory)

        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        
        for file in files:
            imagePath = path+'/'+file
            imageList.append(imagePath)
            labelList.append(label)
    
    return (imageList,labelList)

In [15]:
imageList, labelList = loadDataset()

In [16]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [17]:
features_array = []

print(len(imageList))
print(imageList[1165])

for i in range(len(imageList)):
    if i==1165:
        continue
    print("i = "+str(i))
    img = cv2.imread(imageList[i])
#     print(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = transform(img)
    img = img.reshape(1, 3, 224, 224)
    img = img.to(device)
    with torch.no_grad():
        img_features = feature_extractor(img)
    img_features = img_features.cpu().detach().numpy().reshape(-1)
    features_array.append(img_features)


print(features_array)

3115
/kaggle/input/fruit-and-vegetable-image-recognition/train/bell pepper/Image_56.jpg
i = 0
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
i = 30
i = 31
i = 32
i = 33
i = 34
i = 35
i = 36
i = 37
i = 38
i = 39
i = 40
i = 41
i = 42
i = 43
i = 44
i = 45
i = 46
i = 47
i = 48
i = 49
i = 50
i = 51
i = 52
i = 53
i = 54
i = 55
i = 56
i = 57
i = 58
i = 59
i = 60
i = 61
i = 62
i = 63
i = 64
i = 65
i = 66
i = 67
i = 68
i = 69
i = 70
i = 71
i = 72
i = 73
i = 74
i = 75
i = 76
i = 77
i = 78
i = 79
i = 80
i = 81
i = 82
i = 83
i = 84
i = 85
i = 86
i = 87
i = 88
i = 89
i = 90
i = 91
i = 92
i = 93
i = 94
i = 95
i = 96
i = 97
i = 98
i = 99
i = 100
i = 101
i = 102
i = 103
i = 104
i = 105
i = 106
i = 107
i = 108
i = 109
i = 110
i = 111
i = 112
i = 113
i = 114
i = 115
i = 116
i = 117
i = 118
i = 119
i = 120
i = 121
i = 122
i = 123
i = 124
i = 125
i = 126
i = 12

libpng warning: iCCP: known incorrect sRGB profile


i = 327
i = 328
i = 329
i = 330
i = 331
i = 332
i = 333
i = 334
i = 335
i = 336
i = 337
i = 338
i = 339
i = 340
i = 341
i = 342
i = 343
i = 344
i = 345
i = 346
i = 347
i = 348
i = 349
i = 350
i = 351
i = 352
i = 353
i = 354
i = 355
i = 356
i = 357
i = 358
i = 359
i = 360
i = 361
i = 362
i = 363
i = 364
i = 365
i = 366
i = 367
i = 368
i = 369
i = 370
i = 371
i = 372
i = 373
i = 374
i = 375
i = 376
i = 377
i = 378
i = 379
i = 380
i = 381
i = 382
i = 383
i = 384
i = 385
i = 386
i = 387
i = 388
i = 389
i = 390
i = 391
i = 392
i = 393
i = 394
i = 395
i = 396
i = 397
i = 398
i = 399
i = 400
i = 401
i = 402
i = 403
i = 404
i = 405
i = 406
i = 407
i = 408
i = 409
i = 410
i = 411
i = 412
i = 413
i = 414
i = 415
i = 416
i = 417
i = 418
i = 419
i = 420
i = 421
i = 422
i = 423
i = 424
i = 425
i = 426
i = 427
i = 428
i = 429
i = 430
i = 431
i = 432
i = 433
i = 434
i = 435
i = 436
i = 437
i = 438
i = 439
i = 440
i = 441
i = 442
i = 443
i = 444
i = 445
i = 446
i = 447
i = 448
i = 449
i = 450
i = 451


libpng warning: iCCP: known incorrect sRGB profile


i = 483
i = 484
i = 485
i = 486
i = 487
i = 488
i = 489
i = 490
i = 491
i = 492
i = 493
i = 494
i = 495
i = 496
i = 497
i = 498
i = 499
i = 500
i = 501
i = 502
i = 503
i = 504
i = 505
i = 506
i = 507
i = 508
i = 509
i = 510
i = 511
i = 512
i = 513
i = 514
i = 515
i = 516
i = 517
i = 518
i = 519
i = 520
i = 521
i = 522
i = 523
i = 524
i = 525
i = 526
i = 527
i = 528
i = 529
i = 530
i = 531
i = 532
i = 533
i = 534
i = 535
i = 536
i = 537
i = 538
i = 539
i = 540
i = 541
i = 542
i = 543
i = 544
i = 545
i = 546
i = 547
i = 548
i = 549
i = 550
i = 551
i = 552


...

i = 2709
i = 2710
i = 2711
i = 2712
i = 2713
i = 2714
i = 2715
i = 2716
i = 2717
i = 2718
i = 2719
i = 2720
i = 2721
i = 2722
i = 2723
i = 2724
i = 2725
i = 2726
i = 2727
i = 2728
i = 2729
i = 2730
i = 2731
i = 2732
i = 2733
i = 2734
i = 2735
i = 2736
i = 2737
i = 2738
i = 2739
i = 2740
i = 2741
i = 2742
i = 2743
i = 2744
i = 2745
i = 2746
i = 2747
i = 2748
i = 2749
i = 2750
i = 2751
i = 2752
i = 2753
i = 2754
i = 2755
i = 2756
i = 2757
i = 2758
i = 2759
i = 2760
i = 2761
i = 2762
i = 2763
i = 2764
i = 2765
i = 2766
i = 2767
i = 2768
i = 2769
i = 2770
i = 2771
i = 2772
i = 2773
i = 2774
i = 2775
i = 2776
i = 2777
i = 2778
i = 2779
i = 2780
i = 2781
i = 2782
i = 2783
i = 2784


libpng warning: iCCP: known incorrect sRGB profile


i = 2785
i = 2786
i = 2787
i = 2788
i = 2789
i = 2790
i = 2791
i = 2792
i = 2793
i = 2794
i = 2795
i = 2796
i = 2797
i = 2798
i = 2799
i = 2800
i = 2801
i = 2802
i = 2803
i = 2804
i = 2805
i = 2806
i = 2807


libpng warning: iCCP: known incorrect sRGB profile


i = 2808
i = 2809
i = 2810
i = 2811
i = 2812
i = 2813
i = 2814
i = 2815
i = 2816
i = 2817
i = 2818
i = 2819
i = 2820
i = 2821
i = 2822
i = 2823
i = 2824
i = 2825
i = 2826
i = 2827
i = 2828
i = 2829
i = 2830
i = 2831
i = 2832
i = 2833
i = 2834
i = 2835
i = 2836
i = 2837
i = 2838
i = 2839
i = 2840
i = 2841
i = 2842
i = 2843
i = 2844
i = 2845
i = 2846
i = 2847
i = 2848
i = 2849
i = 2850
i = 2851
i = 2852
i = 2853
i = 2854
i = 2855
i = 2856
i = 2857
i = 2858
i = 2859
i = 2860
i = 2861
i = 2862
i = 2863
i = 2864
i = 2865
i = 2866
i = 2867
i = 2868
i = 2869
i = 2870
i = 2871
i = 2872
i = 2873
i = 2874
i = 2875
i = 2876
i = 2877
i = 2878
i = 2879
i = 2880
i = 2881
i = 2882
i = 2883
i = 2884
i = 2885
i = 2886
i = 2887
i = 2888
i = 2889
i = 2890
i = 2891
i = 2892
i = 2893
i = 2894
i = 2895
i = 2896
i = 2897
i = 2898
i = 2899
i = 2900
i = 2901
i = 2902
i = 2903
i = 2904
i = 2905
i = 2906
i = 2907
i = 2908
i = 2909
i = 2910
i = 2911
i = 2912
i = 2913
i = 2914
i = 2915
i = 2916
i = 2917
i = 2918
i

In [18]:
np.array(features_array).shape

(3114, 25088)

In [19]:
X = np.array(features_array)
del labelList[1165]
y = np.array(labelList)

In [20]:
y.shape
X.shape

(3114, 25088)

In [21]:
df = pd.DataFrame(X)
df['label'] = y

In [22]:
df

,0,1,2,3,4,5,6,7,8,9,...,25079,25080,25081,25082,25083,25084,25085,25086,25087,label
0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,capsicum
1,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,capsicum
2,0.000000,0.0,0.0,0.0,0.477152,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,capsicum
3,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.775946,0.0,0.000000,0.000000,capsicum
4,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,capsicum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,0.211551,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,soy beans
3110,0.052142,0.0,0.0,0.0,0.000000,0.0,3.245011,0.000000,0.635117,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,soy beans
3111,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.895151,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,soy beans
3112,0.000000,0.0,0.0,0.0,0.000000,0.0,0.378726,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.380125,1.700616,soy beans


In [23]:
X=df.drop("label",axis=1)
y=df["label"]
import pickle
with open('X.pkl', 'wb') as file:
    pickle.dump(X, file)   
with open('y.pkl', 'wb') as file:
    pickle.dump(y, file)

In [24]:
import pickle
  
with open('/kaggle/working/X.pkl', 'rb') as file:
    X = pickle.load(file)
with open('/kaggle/working/y.pkl', 'rb') as file:
    y = pickle.load(file)

## Testing

In [25]:
dataset = test_path    # the test path is defined above
imageListTest, labelListTest = loadDataset()

In [26]:
features_array_test = []

print(len(imageListTest))
print(imageListTest[1165])

for i in range(len(imageListTest)):
    if i==1165:
        continue
    print("i = "+str(i))
    img = cv2.imread(imageListTest[i])
#     print(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = transform(img)
    img = img.reshape(1, 3, 224, 224)
    img = img.to(device)
    with torch.no_grad():
        img_features = feature_extractor(img)
    img_features = img_features.cpu().detach().numpy().reshape(-1)
    features_array_test.append(img_features)

3115
/kaggle/input/fruit-and-vegetable-image-recognition/train/bell pepper/Image_56.jpg
i = 0
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
i = 30
i = 31
i = 32
i = 33
i = 34
i = 35
i = 36
i = 37
i = 38
i = 39
i = 40
i = 41
i = 42
i = 43
i = 44
i = 45
i = 46
i = 47
i = 48
i = 49
i = 50
i = 51
i = 52
i = 53
i = 54
i = 55
i = 56
i = 57
i = 58
i = 59
i = 60
i = 61
i = 62
i = 63
i = 64
i = 65
i = 66
i = 67
i = 68
i = 69
i = 70
i = 71
i = 72
i = 73
i = 74
i = 75
i = 76
i = 77
i = 78
i = 79
i = 80
i = 81
i = 82
i = 83
i = 84
i = 85
i = 86
i = 87
i = 88
i = 89
i = 90
i = 91
i = 92
i = 93
i = 94
i = 95
i = 96
i = 97
i = 98
i = 99
i = 100
i = 101
i = 102
i = 103
i = 104
i = 105
i = 106
i = 107
i = 108
i = 109
i = 110
i = 111
i = 112
i = 113
i = 114
i = 115
i = 116
i = 117
i = 118
i = 119
i = 120
i = 121
i = 122
i = 123
i = 124
i = 125
i = 126
i = 12

libpng warning: iCCP: known incorrect sRGB profile


i = 327
i = 328
i = 329
i = 330
i = 331
i = 332
i = 333
i = 334
i = 335
i = 336
i = 337
i = 338
i = 339
i = 340
i = 341
i = 342
i = 343
i = 344
i = 345
i = 346
i = 347
i = 348
i = 349
i = 350
i = 351
i = 352
i = 353
i = 354
i = 355
i = 356
i = 357
i = 358
i = 359
i = 360
i = 361
i = 362
i = 363
i = 364
i = 365
i = 366
i = 367
i = 368
i = 369
i = 370
i = 371
i = 372
i = 373
i = 374
i = 375
i = 376
i = 377
i = 378
i = 379
i = 380
i = 381
i = 382
i = 383
i = 384
i = 385
i = 386
i = 387
i = 388
i = 389
i = 390
i = 391
i = 392
i = 393
i = 394
i = 395
i = 396
i = 397
i = 398
i = 399
i = 400
i = 401
i = 402
i = 403
i = 404
i = 405
i = 406
i = 407
i = 408
i = 409
i = 410
i = 411
i = 412
i = 413
i = 414
i = 415
i = 416
i = 417
i = 418
i = 419
i = 420
i = 421
i = 422
i = 423
i = 424
i = 425
i = 426
i = 427
i = 428
i = 429
i = 430
i = 431
i = 432
i = 433
i = 434
i = 435
i = 436
i = 437
i = 438
i = 439
i = 440
i = 441
i = 442
i = 443
i = 444
i = 445
i = 446
i = 447
i = 448
i = 449
i = 450
i = 451


libpng warning: iCCP: known incorrect sRGB profile


i = 483
i = 484
i = 485
i = 486
i = 487
i = 488
i = 489
i = 490
i = 491
i = 492
i = 493
i = 494
i = 495
i = 496
i = 497
i = 498
i = 499
i = 500
i = 501
i = 502
i = 503
i = 504
i = 505
i = 506
i = 507
i = 508
i = 509
i = 510
i = 511
i = 512
i = 513
i = 514
i = 515
i = 516
i = 517
i = 518
i = 519
i = 520
i = 521
i = 522
i = 523
i = 524
i = 525
i = 526
i = 527
i = 528
i = 529
i = 530
i = 531
i = 532
i = 533
i = 534
i = 535
i = 536
i = 537
i = 538
i = 539
i = 540
i = 541
i = 542
i = 543
i = 544
i = 545
i = 546
i = 547
i = 548
i = 549
i = 550
i = 551
i = 552
i = 553
i = 554
i = 555


...

i = 2711
i = 2712
i = 2713
i = 2714
i = 2715
i = 2716
i = 2717
i = 2718
i = 2719
i = 2720
i = 2721
i = 2722
i = 2723
i = 2724
i = 2725
i = 2726
i = 2727
i = 2728
i = 2729
i = 2730
i = 2731
i = 2732
i = 2733
i = 2734
i = 2735
i = 2736
i = 2737
i = 2738
i = 2739
i = 2740
i = 2741
i = 2742
i = 2743
i = 2744
i = 2745
i = 2746
i = 2747
i = 2748
i = 2749
i = 2750
i = 2751
i = 2752
i = 2753
i = 2754
i = 2755
i = 2756
i = 2757
i = 2758
i = 2759
i = 2760
i = 2761
i = 2762
i = 2763
i = 2764
i = 2765
i = 2766
i = 2767
i = 2768
i = 2769
i = 2770
i = 2771
i = 2772
i = 2773
i = 2774
i = 2775
i = 2776
i = 2777
i = 2778
i = 2779
i = 2780
i = 2781
i = 2782
i = 2783
i = 2784
i = 2785
i = 2786


libpng warning: iCCP: known incorrect sRGB profile


i = 2787
i = 2788
i = 2789
i = 2790
i = 2791
i = 2792
i = 2793
i = 2794
i = 2795
i = 2796
i = 2797
i = 2798
i = 2799
i = 2800
i = 2801
i = 2802
i = 2803
i = 2804
i = 2805
i = 2806
i = 2807
i = 2808


libpng warning: iCCP: known incorrect sRGB profile


i = 2809
i = 2810
i = 2811
i = 2812
i = 2813
i = 2814
i = 2815
i = 2816
i = 2817
i = 2818
i = 2819
i = 2820
i = 2821
i = 2822
i = 2823
i = 2824
i = 2825
i = 2826
i = 2827
i = 2828
i = 2829
i = 2830
i = 2831
i = 2832
i = 2833
i = 2834
i = 2835
i = 2836
i = 2837
i = 2838
i = 2839
i = 2840
i = 2841
i = 2842
i = 2843
i = 2844
i = 2845
i = 2846
i = 2847
i = 2848
i = 2849
i = 2850
i = 2851
i = 2852
i = 2853
i = 2854
i = 2855
i = 2856
i = 2857
i = 2858
i = 2859
i = 2860
i = 2861
i = 2862
i = 2863
i = 2864
i = 2865
i = 2866
i = 2867
i = 2868
i = 2869
i = 2870
i = 2871
i = 2872
i = 2873
i = 2874
i = 2875
i = 2876
i = 2877
i = 2878
i = 2879
i = 2880
i = 2881
i = 2882
i = 2883
i = 2884
i = 2885
i = 2886
i = 2887
i = 2888
i = 2889
i = 2890
i = 2891
i = 2892
i = 2893
i = 2894
i = 2895
i = 2896
i = 2897
i = 2898
i = 2899
i = 2900
i = 2901
i = 2902
i = 2903
i = 2904
i = 2905
i = 2906
i = 2907
i = 2908
i = 2909
i = 2910
i = 2911
i = 2912
i = 2913
i = 2914
i = 2915
i = 2916
i = 2917
i = 2918
i = 2919
i

In [28]:
X_test = np.array(features_array_test)
del labelListTest[1165]
y_test = np.array(labelListTest)
print(len(y_test))

X_test.shape

3114


(3114, 25088)

In [30]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train = X
y_train = y

X_train.shape
y_train.shape
y.shape


(3114,)

In [31]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=100000).fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test,y_test)

0.9942196531791907

In [32]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='linear'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.9942196531791907

In [33]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='rbf'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.9826589595375722

In [32]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
neigh.score(X_test,y_test)

0.6143224149004496

In [36]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42,n_estimators=500)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.9942196531791907

In [37]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.9563262684649968

## Bikes vs Horses Dataset

In [33]:
from imutils import paths
import os
import cv2
import numpy as np
import pandas as pd
import torch

In [34]:
images = []
labels = []

In [35]:
image_paths = list(paths.list_images('/kaggle/input/bikesandhorses/Horses'))
print(len(image_paths))

99


In [36]:
for img_path in image_paths:
    label = 'horse'
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    labels.append(label)
    
print(len(images))

99


In [37]:
image_paths = list(paths.list_images('/kaggle/input/bikesandhorses/Bikes'))

In [38]:
for img_path in image_paths:
    label = 'bike'
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    labels.append(label)

In [39]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [40]:
features_array = []

for i in range(len(images)):
    img = images[i]
    img = transform(img)
    img = img.reshape(1, 3, 224, 224)
    img = img.to(device)
    with torch.no_grad():
        img_features = feature_extractor(img)
    img_features = img_features.cpu().detach().numpy().reshape(-1)
    features_array.append(img_features)

In [41]:
np.array(features_array).shape

(179, 25088)

In [42]:
X = np.array(features_array)
y = np.array(labels)

In [43]:
df = pd.DataFrame(X)
df['label'] = y

In [44]:
X=df.drop("label",axis=1)
y=df["label"]
import pickle
with open('X_bikevshorse.pkl', 'wb') as file:
    pickle.dump(X, file)
with open('y_bikevshorse.pkl', 'wb') as file:
    pickle.dump(y, file)

In [45]:
import pickle

with open('X_bikevshorse.pkl', 'rb') as file:
    X = pickle.load(file)
with open('y_bikevshorse.pkl', 'rb') as file:
    y = pickle.load(file)

In [46]:
df

,0,1,2,3,4,5,6,7,8,9,...,25079,25080,25081,25082,25083,25084,25085,25086,25087,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,2.076342,0.139766,0.000000,0.000000,horse
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,horse
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.152882,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,horse
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.065098,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,horse
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.913381,0.0,0.0,0.0,0.0,0.000000,0.000000,1.327906,3.405962,horse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,bike
175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,bike
176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,bike
177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,bike


In [47]:
X.shape

(179, 25088)

In [48]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_train.shape

(89, 25088)

In [72]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=100000).fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test,y_test)

1.0

In [73]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='linear'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.9222222222222223

In [74]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='rbf'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.6333333333333333

In [75]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

1.0

In [76]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

1.0

In [77]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=50, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

1.0

In [78]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42,n_estimators=500)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

1.0

In [59]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(X_train, y_train)
neigh.score(X_test,y_test)

1.0